### OPEN CLIP

In [ ]:
import open_clip

In [ ]:
model, _, _ = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')

In [3]:
visual_encoder = model.visual
text_encoder = model.transformer

In [4]:
print(visual_encoder)

VisionTransformer(
  (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-23): 24 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
)


In [5]:
print(text_encoder)

Transformer(
  (resblocks): ModuleList(
    (0-11): 12 x ResidualAttentionBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
      (ls_1): Identity()
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (gelu): QuickGELU()
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
      )
      (ls_2): Identity()
    )
  )
)


In [8]:
model.encode_image()
model.encode_text()

TypeError: encode_text() missing 1 required positional argument: 'text'

In [1]:
import torch
import torchvision
import open_clip

/home/gridsan/manderson/.conda/envs/train-clip/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print(torch.__version__)

2.4.1+cu121


In [7]:
print(torch.cuda.is_available())

True


### CLIP

In [3]:
from pytorch_lightning import Trainer

In [4]:
trainer = Trainer()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/gridsan/manderson/.conda/envs/train-clip/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [1]:
from transformers import CLIPModel, CLIPTokenizer

/home/gridsan/manderson/.conda/envs/train-clip/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = CLIPModel.from_pretrained("/home/gridsan/manderson/ovdsat/weights/clip-vit-large-patch14")

In [4]:
tokenizer = CLIPTokenizer.from_pretrained("/home/gridsan/manderson/ovdsat/weights/clip-vit-large-patch14")

/home/gridsan/manderson/.conda/envs/train-clip/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
print(model.text_model)
print()
print(model.vision_model)

CLIPTextTransformer(
  (embeddings): CLIPTextEmbeddings(
    (token_embedding): Embedding(49408, 768)
    (position_embedding): Embedding(77, 768)
  )
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-11): 12 x CLIPEncoderLayer(
        (self_attn): CLIPSdpaAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (final_layer_norm

In [28]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Layer {name} is frozen.")
    else:
        print(f"Layer {name} is trainable.")
print()

Layer logit_scale is trainable.
Layer text_model.embeddings.token_embedding.weight is trainable.
Layer text_model.embeddings.position_embedding.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.k_proj.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.k_proj.bias is trainable.
Layer text_model.encoder.layers.0.self_attn.v_proj.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.v_proj.bias is trainable.
Layer text_model.encoder.layers.0.self_attn.q_proj.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.q_proj.bias is trainable.
Layer text_model.encoder.layers.0.self_attn.out_proj.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.out_proj.bias is trainable.
Layer text_model.encoder.layers.0.layer_norm1.weight is trainable.
Layer text_model.encoder.layers.0.layer_norm1.bias is trainable.
Layer text_model.encoder.layers.0.mlp.fc1.weight is trainable.
Layer text_model.encoder.layers.0.mlp.fc1.bias is trainable.
Layer t

In [34]:
for name, param in custom_model.named_parameters():
    if not param.requires_grad:
        print(f"Layer {name} is frozen.")
    else:
        print(f"Layer {name} is trainable.")
print()

Layer sink_temp is trainable.
Layer model.positional_embedding is trainable.
Layer model.text_projection is trainable.
Layer model.logit_scale is trainable.
Layer model.token_embedding.weight is trainable.
Layer model.ln_final.weight is trainable.
Layer model.ln_final.bias is trainable.
Layer model.visual_projection.weight is trainable.
Layer model.visual_projection.bias is trainable.
Layer model.visual.embeddings.class_embedding is trainable.
Layer model.visual.embeddings.patch_embedding.weight is trainable.
Layer model.visual.embeddings.position_embedding.weight is trainable.
Layer model.visual.pre_layrnorm.weight is trainable.
Layer model.visual.pre_layrnorm.bias is trainable.
Layer model.visual.encoder.layers.0.self_attn.k_proj.weight is trainable.
Layer model.visual.encoder.layers.0.self_attn.k_proj.bias is trainable.
Layer model.visual.encoder.layers.0.self_attn.v_proj.weight is trainable.
Layer model.visual.encoder.layers.0.self_attn.v_proj.bias is trainable.
Layer model.visual.

In [29]:
print(model.visual_projection.weight.shape)

torch.Size([768, 1024])


In [33]:
print(model.visual_projection.weight[0,0])

tensor(0.0224, grad_fn=<SelectBackward0>)


In [7]:
print(model.text_projection.weight.shape)

torch.Size([768, 768])


In [8]:
print(type(model))

<class 'transformers.models.clip.modeling_clip.CLIPModel'>


In [10]:
print(type(model.vision_model))

<class 'transformers.models.clip.modeling_clip.CLIPVisionTransformer'>


In [11]:
print(model.vision_model)

CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(257, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, 

In [12]:
for name, param in model.vision_model.named_parameters():
    if not param.requires_grad:
        print(f"Layer {name} is frozen.")
    else:
        print(f"Layer {name} is trainable.")
print()

Layer embeddings.class_embedding is trainable.
Layer embeddings.patch_embedding.weight is trainable.
Layer embeddings.position_embedding.weight is trainable.
Layer pre_layrnorm.weight is trainable.
Layer pre_layrnorm.bias is trainable.
Layer encoder.layers.0.self_attn.k_proj.weight is trainable.
Layer encoder.layers.0.self_attn.k_proj.bias is trainable.
Layer encoder.layers.0.self_attn.v_proj.weight is trainable.
Layer encoder.layers.0.self_attn.v_proj.bias is trainable.
Layer encoder.layers.0.self_attn.q_proj.weight is trainable.
Layer encoder.layers.0.self_attn.q_proj.bias is trainable.
Layer encoder.layers.0.self_attn.out_proj.weight is trainable.
Layer encoder.layers.0.self_attn.out_proj.bias is trainable.
Layer encoder.layers.0.layer_norm1.weight is trainable.
Layer encoder.layers.0.layer_norm1.bias is trainable.
Layer encoder.layers.0.mlp.fc1.weight is trainable.
Layer encoder.layers.0.mlp.fc1.bias is trainable.
Layer encoder.layers.0.mlp.fc2.weight is trainable.
Layer encoder.la

In [13]:
model.vision_model.visual_projection = model.visual_projection

In [14]:
for name, param in model.vision_model.named_parameters():
    print(name)

embeddings.class_embedding
embeddings.patch_embedding.weight
embeddings.position_embedding.weight
pre_layrnorm.weight
pre_layrnorm.bias
encoder.layers.0.self_attn.k_proj.weight
encoder.layers.0.self_attn.k_proj.bias
encoder.layers.0.self_attn.v_proj.weight
encoder.layers.0.self_attn.v_proj.bias
encoder.layers.0.self_attn.q_proj.weight
encoder.layers.0.self_attn.q_proj.bias
encoder.layers.0.self_attn.out_proj.weight
encoder.layers.0.self_attn.out_proj.bias
encoder.layers.0.layer_norm1.weight
encoder.layers.0.layer_norm1.bias
encoder.layers.0.mlp.fc1.weight
encoder.layers.0.mlp.fc1.bias
encoder.layers.0.mlp.fc2.weight
encoder.layers.0.mlp.fc2.bias
encoder.layers.0.layer_norm2.weight
encoder.layers.0.layer_norm2.bias
encoder.layers.1.self_attn.k_proj.weight
encoder.layers.1.self_attn.k_proj.bias
encoder.layers.1.self_attn.v_proj.weight
encoder.layers.1.self_attn.v_proj.bias
encoder.layers.1.self_attn.q_proj.weight
encoder.layers.1.self_attn.q_proj.bias
encoder.layers.1.self_attn.out_proj.

### Load checkpoint

In [1]:
import torch
from transformers import CLIPModel
from models import CustomCLIPWrapper

/home/gridsan/manderson/.conda/envs/train-clip/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
PATH_CKPT_clip14_test = '/home/gridsan/manderson/train-CLIP/run/test/last.ckpt'

In [25]:
ckpt = torch.load(PATH_CKPT_clip14_test, map_location='cpu')

/state/partition1/slurm_tmp/27105912.0.0/ipykernel_2330604/3157462980.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(PATH_CKPT_clip14_test, map_locati

In [26]:
print(ckpt.keys())

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'callbacks', 'optimizer_states', 'lr_schedulers', 'native_amp_scaling_state'])


In [5]:
state_dict = ckpt['state_dict']

In [6]:
print(state_dict.keys())

odict_keys(['sink_temp', 'model.positional_embedding', 'model.text_projection', 'model.logit_scale', 'model.token_embedding.weight', 'model.ln_final.weight', 'model.ln_final.bias', 'model.visual_projection.weight', 'model.visual.embeddings.class_embedding', 'model.visual.embeddings.patch_embedding.weight', 'model.visual.embeddings.position_embedding.weight', 'model.visual.pre_layrnorm.weight', 'model.visual.pre_layrnorm.bias', 'model.visual.encoder.layers.0.self_attn.k_proj.weight', 'model.visual.encoder.layers.0.self_attn.k_proj.bias', 'model.visual.encoder.layers.0.self_attn.v_proj.weight', 'model.visual.encoder.layers.0.self_attn.v_proj.bias', 'model.visual.encoder.layers.0.self_attn.q_proj.weight', 'model.visual.encoder.layers.0.self_attn.q_proj.bias', 'model.visual.encoder.layers.0.self_attn.out_proj.weight', 'model.visual.encoder.layers.0.self_attn.out_proj.bias', 'model.visual.encoder.layers.0.layer_norm1.weight', 'model.visual.encoder.layers.0.layer_norm1.bias', 'model.visual.e

In [7]:
model = CLIPModel.from_pretrained('/home/gridsan/manderson/ovdsat/weights/clip-vit-large-patch14')

In [8]:
model.vision_model 

CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(257, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, 

In [9]:
model.text_model

CLIPTextTransformer(
  (embeddings): CLIPTextEmbeddings(
    (token_embedding): Embedding(49408, 768)
    (position_embedding): Embedding(77, 768)
  )
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-11): 12 x CLIPEncoderLayer(
        (self_attn): CLIPSdpaAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (final_layer_norm

In [27]:
# Init custom clip model
custom_model = CustomCLIPWrapper(
    model.vision_model,
    model.text_model,
    minibatch_size=0
)

In [28]:
custom_model.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [32]:
print(model.text_projection.weight.shape)
print(model.visual_projection.weight.shape)
print(model.text_projection.weight[0,0])

torch.Size([768, 768])
torch.Size([768, 1024])
tensor(-0.0109, grad_fn=<SelectBackward0>)


In [ ]:
print(custom_model

In [13]:
print(custom_model.model.visual)

CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(257, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, 

In [22]:
model.vision_model = custom_model.model.visual
model.text_model = custom_model.model.transformer

In [23]:
print(model.vision_model)

CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(257, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, 

In [16]:
i = 0
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Layer {name} is frozen.")
    else:
        print(f"Layer {name} is trainable.")
    i+=1
print(f'{i} total params')

Layer logit_scale is trainable.
Layer text_model.embeddings.token_embedding.weight is trainable.
Layer text_model.embeddings.position_embedding.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.k_proj.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.k_proj.bias is trainable.
Layer text_model.encoder.layers.0.self_attn.v_proj.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.v_proj.bias is trainable.
Layer text_model.encoder.layers.0.self_attn.q_proj.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.q_proj.bias is trainable.
Layer text_model.encoder.layers.0.self_attn.out_proj.weight is trainable.
Layer text_model.encoder.layers.0.self_attn.out_proj.bias is trainable.
Layer text_model.encoder.layers.0.layer_norm1.weight is trainable.
Layer text_model.encoder.layers.0.layer_norm1.bias is trainable.
Layer text_model.encoder.layers.0.mlp.fc1.weight is trainable.
Layer text_model.encoder.layers.0.mlp.fc1.bias is trainable.
Layer t

In [34]:
print(model.text_projection.weight.shape)
print(model.visual_projection.weight.shape)

torch.Size([768, 768])
torch.Size([768, 1024])


In [35]:
print(custom_model.model.text_projection[0,0])

tensor(0.0522, grad_fn=<SelectBackward0>)


In [15]:
i = 0
for name, param in custom_model.named_parameters():
    if not param.requires_grad:
        print(f"Layer {name} is frozen.")
    else:
        print(f"Layer {name} is trainable.")
    i+=1
print(f'{i} total params')

Layer sink_temp is trainable.
Layer model.positional_embedding is trainable.
Layer model.text_projection is trainable.
Layer model.logit_scale is trainable.
Layer model.token_embedding.weight is trainable.
Layer model.ln_final.weight is trainable.
Layer model.ln_final.bias is trainable.
Layer model.visual_projection.weight is trainable.
Layer model.visual.embeddings.class_embedding is trainable.
Layer model.visual.embeddings.patch_embedding.weight is trainable.
Layer model.visual.embeddings.position_embedding.weight is trainable.
Layer model.visual.pre_layrnorm.weight is trainable.
Layer model.visual.pre_layrnorm.bias is trainable.
Layer model.visual.encoder.layers.0.self_attn.k_proj.weight is trainable.
Layer model.visual.encoder.layers.0.self_attn.k_proj.bias is trainable.
Layer model.visual.encoder.layers.0.self_attn.v_proj.weight is trainable.
Layer model.visual.encoder.layers.0.self_attn.v_proj.bias is trainable.
Layer model.visual.encoder.layers.0.self_attn.q_proj.weight is train

### Convert to CLIP

In [4]:
import torch
from transformers import CLIPModel
from models import CustomCLIPWrapper
import glob

In [7]:
model_dir = '/home/gridsan/manderson/train-CLIP/run/train'
clip_pretrain = '/home/gridsan/manderson/ovdsat/weights/clip-vit-large-patch14'

In [5]:
ckpt_files = glob.glob(f'{model_dir}/*')

In [6]:
print(ckpt_files)

['/home/gridsan/manderson/train-CLIP/run/train/last.ckpt', '/home/gridsan/manderson/train-CLIP/run/train/model-epoch=24-val_loss=1.00.ckpt', '/home/gridsan/manderson/train-CLIP/run/train/model-epoch=20-val_loss=1.00.ckpt', '/home/gridsan/manderson/train-CLIP/run/train/model-epoch=33-val_loss=0.99.ckpt']


In [ ]:
# Load clip model
clip_model = CLIPModel.from_pretrained(clip_pretrain)
clip_keys = list(clip_model.state_dict().keys())
print(f'{len(clip_keys)} parameters in model')

In [28]:
for custom_ckpt in ckpt_files:
    print(f'Converting {custom_ckpt}...')
    convert_name = custom_ckpt.split('.')[0]+'_clip.pth'
    
    # Load custom model parameters
    ckpt = torch.load(custom_ckpt, map_location='cpu')
    custom_state_dict = ckpt['state_dict']
    custom_keys = list(custom_state_dict.keys())
    
    # Convert custom to clip
    convert_state_dict = {}

    i = 0
    for k in clip_keys:
        parts = k.split('.')
        if len(parts) <= 2:
            i+=1
            custom_name = [name for name in custom_keys if parts[0] in name][0]
        elif parts[0] == 'text_model':
            i+=1
            custom_name = 'model.transformer.'+'.'.join(parts[1:])
        elif parts[0] == 'vision_model':
            i+=1
            custom_name = 'model.visual.'+'.'.join(parts[1:])
        v = custom_state_dict[custom_name]
        convert_state_dict[k] = v
    print(f'{i} parameters converted')
    
    # Check compatibility
    ret = clip_model.load_state_dict(convert_state_dict)
    all_keys_matched = len(ret.missing_keys) == 0 and len(ret.unexpected_keys) == 0
    
    # If successful, save new state dict
    if ret:
        torch.save(convert_state_dict, convert_name)
        print(f'Saved converted model to {convert_name}')

Converting /home/gridsan/manderson/train-CLIP/run/train/last.ckpt...


/state/partition1/slurm_tmp/27123069.0.0/ipykernel_1068323/333349102.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(custom_ckpt, map_location='cpu')


590 parameters in model
590 parameters converted
Saved converted model to /home/gridsan/manderson/train-CLIP/run/train/last_cvt.pth
Converting /home/gridsan/manderson/train-CLIP/run/train/model-epoch=24-val_loss=1.00.ckpt...
590 parameters in model
590 parameters converted
Saved converted model to /home/gridsan/manderson/train-CLIP/run/train/model-epoch=24-val_loss=1_cvt.pth
Converting /home/gridsan/manderson/train-CLIP/run/train/model-epoch=20-val_loss=1.00.ckpt...
590 parameters in model
590 parameters converted
Saved converted model to /home/gridsan/manderson/train-CLIP/run/train/model-epoch=20-val_loss=1_cvt.pth
Converting /home/gridsan/manderson/train-CLIP/run/train/model-epoch=33-val_loss=0.99.ckpt...
590 parameters in model
590 parameters converted
Saved converted model to /home/gridsan/manderson/train-CLIP/run/train/model-epoch=33-val_loss=0_cvt.pth


#### one at a time

In [10]:
custom_ckpt = '/home/gridsan/manderson/train-CLIP/run/train/last.ckpt'
clip_pretrain = '/home/gridsan/manderson/ovdsat/weights/clip-vit-large-patch14'
convert_name = custom_ckpt.split('.')[0]+'_cvt.pth'
print(convert_name)

/home/gridsan/manderson/train-CLIP/run/train/last_cvt.pth


In [11]:
# Load custom model parameters
ckpt = torch.load(custom_ckpt, map_location='cpu')
custom_state_dict = ckpt['state_dict']
custom_keys = list(custom_state_dict.keys())

/state/partition1/slurm_tmp/27123069.0.0/ipykernel_1068323/2380601795.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(custom_ckpt, map_location='cpu')


In [12]:
# Load clip model
clip_model = CLIPModel.from_pretrained(clip_pretrain)
clip_keys = list(clip_model.state_dict().keys())
print(f'{len(clip_keys)} parameters in model')

590 parameters in model


In [15]:
# Convert custom to clip
convert_state_dict = {}

i = 0
for k in clip_keys:
    parts = k.split('.')
    if len(parts) <= 2:
        i+=1
        custom_name = [name for name in custom_keys if parts[0] in name][0]
    elif parts[0] == 'text_model':
        i+=1
        custom_name = 'model.transformer.'+'.'.join(parts[1:])
    elif parts[0] == 'vision_model':
        i+=1
        custom_name = 'model.visual.'+'.'.join(parts[1:])
    v = custom_state_dict[custom_name]
    convert_state_dict[k] = v
print(f'{i} parameters converted')

590 parameters converted


In [16]:
# Check compatibility
print(clip_model.load_state_dict(convert_state_dict))

<All keys matched successfully>


In [17]:
ret = clip_model.load_state_dict(convert_state_dict)

In [20]:
print(type(ret))

<class 'torch.nn.modules.module._IncompatibleKeys'>


In [22]:
all_keys_matched = len(ret.missing_keys) == 0 and len(ret.unexpected_keys) == 0
print(all_keys_matched)

True


In [96]:
# If successful, save new state dict
torch.save(convert_state_dict, convert_name)

In [98]:
convert_name

'/home/gridsan/manderson/train-CLIP/run/test/last_cvt.pth'

In [100]:
model = CLIPModel.from_pretrained("/home/gridsan/manderson/ovdsat/weights/clip-vit-large-patch14")
ckpt = torch.load(convert_name, map_location="cpu")
model.load_state_dict(ckpt)
model = model.vision_model
model.output_tokens = True
print(f'Using checkpoint {convert_name}')

/state/partition1/slurm_tmp/27105912.0.0/ipykernel_2330604/2724428146.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(convert_name, map_location="cpu")

Using checkpoint /home/gridsan/manderson/train-CLIP/run/test/last_cvt.pth
